## 1. Import Necessary Libraries

In [13]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import metrics
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D

## 2. Load Data

In [2]:
# Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')

# Generate file paths correctly
train_df['file_path'] = train_df.apply(
    lambda row: f"/kaggle/input/bttai-ajl-2025/train/train/{row['label']}/{row['md5hash']}.jpg", axis=1
)
test_df['file_path'] = test_df['md5hash'].apply(
    lambda x: f"/kaggle/input/bttai-ajl-2025/test/test/{x}.jpg"
)

# Remove invalid rows
train_df = train_df[(train_df['fitzpatrick_scale'] > 0) & (train_df['label'].notna())]
train_df = train_df[train_df['file_path'].apply(os.path.exists)]
test_df = test_df[test_df['file_path'].apply(os.path.exists)]


## 3. Data Preprocessing

In [4]:
# Data Preprocessing

# Encode the labels
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])


# Splitting dataset into training and validation datasets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['encoded_label'])



# Define image data generators for training and testing
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.9, 1.1],
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='file_path',
    y_col='encoded_label',
    target_size=(224, 224),
    batch_size=128,
    class_mode='raw',
    shuffle = True
)


val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='file_path',
    target_size=(224, 224),
    batch_size=128,
    class_mode=None,
    shuffle=False
    
)


test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='file_path',
    target_size=(224, 224),
    batch_size= 128,
    class_mode=None,
    shuffle=False
    
)

# Compute Class Weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df['encoded_label']),
    y=train_df['encoded_label']
)
class_weights_dict = dict(enumerate(class_weights))

Found 2201 validated image filenames.
Found 551 validated image filenames.
Found 1227 validated image filenames.


## 4. Model Training

In [21]:

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers[:200]:  
    layer.trainable = False  



efficientnetb0 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(21, activation='softmax')  # Adjust number of classes
])

efficientnetb0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

efficientnetb0.fit(train_generator, epochs=15)

Epoch 1/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.1797 - loss: 2.7998
Epoch 2/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.4291 - loss: 1.8902
Epoch 3/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5609 - loss: 1.4134
Epoch 4/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.6686 - loss: 1.0612
Epoch 5/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.7851 - loss: 0.7720
Epoch 6/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.7982 - loss: 0.6234
Epoch 7/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.8465 - loss: 0.5212
Epoch 8/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.8778 - loss: 0.3978
Epoch 9/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9225 - loss: 0.2759
Epoch 10/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9159 - loss: 0.2828
Epoch 11/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9243 - loss: 0.2486
Epoch 12/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9427 - lo

In [25]:

# Generate predictions
# y_pred_res = np.argmax(resnet50.predict(val_generator), axis=1) 
y_pred_eff = np.argmax(efficientnetb0.predict(val_generator), axis=1)
y_true = val_df['encoded_label'].values 

# Calculate F1 Score
# f1 = f1_score(y_true, y_pred_res, average='weighted')
# print("ResNet 50 F1 Score:", f1)

f1 = f1_score(y_true, y_pred_eff, average='weighted')
print("Efficient Net b0 F1 Score:", f1)


# # SUBMISSION.CSV
y_pred = np.argmax(efficientnetb0.predict(test_generator), axis = 1)
test_df['label'] = label_encoder.inverse_transform(y_pred)

# Save submission
test_df[['md5hash', 'label']].to_csv('/kaggle/working/submission.csv', index=False)

5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 518ms/step
Efficient Net b0 F1 Score: 0.5449766222577818


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step
